#### k-folds 交叉验证 把训练数据集分成k份，称为k-folds cross validation
#### 缺点，每次训练k个模型，相当于整体性能慢了了k倍

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [2]:
digits = datasets.load_digits()
X, y = digits.data, digits.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

### 网格搜索

In [4]:
best_score, best_p, best_k = 0, 0, 0 
for k in range(1, 11):
    for p in range(1, 6):
        knn = KNeighborsClassifier(n_neighbors=k, p=p, weights="distance")
        knn.fit(X_train, y_train)
        score = knn.score(X_test, y_test)
        
        if score > best_score:
            best_score, best_p, best_k = score, p, k
            
print(best_score, best_p, best_k)

0.9911111111111112 3 1


### 交叉验证

In [5]:
## 1.cross_val_score(X_train),
## 2.get best_k, best_p
## 3.get best estimator
## 4.score X_test
best_score, best_p, best_k = 0, 0, 0 
for k in range(1, 11):
    for p in range(1, 6):
        knn = KNeighborsClassifier(n_neighbors=k, p=p, weights="distance")
        score = np.mean(cross_val_score(knn, X_train, y_train))
        
        if score > best_score:
            best_score, best_p, best_k = score, p, k
            
print(best_score, best_p, best_k)

0.9859011427784662 3 4


In [6]:
knn = KNeighborsClassifier(n_neighbors=4, p=3, weights="distance")
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.9911111111111112

### GridSearchCV

In [7]:
param_grid = {'n_neighbors': [i for i in range(1, 11)], 
              'p': [i for i in range(1, 6)], 
              'weights': ('distance', 'uniform')}

In [8]:
grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, verbose=1)

In [12]:
## 采用交叉验证
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'p': [1, 2, 3, 4, 5],
                         'weights': ('distance', 'uniform')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [14]:
grid.best_score_

0.9859011427784662

In [10]:
grid.best_params_

{'n_neighbors': 4, 'p': 3, 'weights': 'distance'}

In [11]:
estimator = grid.best_estimator_
estimator.score(X_test, y_test)

0.9911111111111112